##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [ ]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 199 kB 54.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=4cbcea530393d1a57507d4b0c9613e2acc693a41107a64df580a15c326123378
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [ ]:
data= spark.read.load("SMSSpamCollection",format="csv", sep="\t", inferSchema="true")

### Print the schema

In [ ]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [ ]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



### Rename the first column to 'class' and second column to 'text'

In [ ]:
data = data.withColumnRenamed("_c0",'class')
data = data.withColumnRenamed("_c1", 'text')

In [ ]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [ ]:
data.show(10,truncate=True)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
data.show(10 , truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [ ]:
from pyspark.sql.functions import length
data = data.withColumn('length_of_text', length(data.text))


### Show the new dataframe

In [ ]:
data.show()

+-----+--------------------+--------------+
|class|                text|length_of_text|
+-----+--------------------+--------------+
|  ham|Go until jurong p...|           111|
|  ham|Ok lar... Joking ...|            29|
| spam|Free entry in 2 a...|           155|
|  ham|U dun say so earl...|            49|
|  ham|Nah I don't think...|            61|
| spam|FreeMsg Hey there...|           147|
|  ham|Even my brother i...|            77|
|  ham|As per your reque...|           160|
| spam|WINNER!! As a val...|           157|
| spam|Had your mobile 1...|           154|
|  ham|I'm gonna be home...|           109|
| spam|SIX chances to wi...|           136|
| spam|URGENT! You have ...|           155|
|  ham|I've been searchi...|           196|
|  ham|I HAVE A DATE ON ...|            35|
| spam|XXXMobileMovieClu...|           149|
|  ham|Oh k...i'm watchi...|            26|
|  ham|Eh u remember how...|            81|
|  ham|Fine if thats th...|            56|
| spam|England v Macedon...|    

### Get the average text length for each class (give alias name to the average length column)

In [ ]:
data.createOrReplaceTempView("TempView")


In [ ]:
spark.sql("select  class , avg(length_of_text) from TempView group By(class)").show()

+-----+-------------------+
|class|avg(length_of_text)|
+-----+-------------------+
|  ham|  71.45431945307645|
| spam|  138.6706827309237|
+-----+-------------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [ ]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer


In [ ]:
tkn = Tokenizer(inputCol="text", outputCol="Tokenize")
stopwords = StopWordsRemover(inputCol='Tokenize',outputCol='stopWords')
count_vec = CountVectorizer(inputCol = "stopWords" , outputCol="CountVectorizer")
idf = IDF(inputCol="CountVectorizer", outputCol="TF-IDF")

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [ ]:
from pyspark.ml.feature import StringIndexer,VectorAssembler

labels = StringIndexer(inputCol='class',outputCol='label')
features = VectorAssembler(inputCols=['TF-IDF','length_of_text'],outputCol='features')


## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [ ]:
from pyspark.ml.classification import NaiveBayes
model = NaiveBayes()

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [ ]:
from pyspark.ml import Pipeline
pipe=Pipeline(stages=[labels,tkn,stopwords,count_vec,idf,features,model])



### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [ ]:

train_df,test_df = data.randomSplit([0.7, 0.3],15)
train_df.show()

+-----+--------------------+--------------+
|class|                text|length_of_text|
+-----+--------------------+--------------+
|  ham| &lt;#&gt;  in mc...|            36|
|  ham| &lt;#&gt;  mins ...|            51|
|  ham| &lt;DECIMAL&gt; ...|           132|
|  ham| and  picking the...|            41|
|  ham| gonna let me kno...|            95|
|  ham| says that he's q...|           200|
|  ham| what number do u...|            36|
|  ham|"Gimme a few" was...|            41|
|  ham|"Happy valentines...|           147|
|  ham|"Its Ur luck to L...|           155|
|  ham|"Life is nothing ...|           159|
|  ham|"Response" is one...|           154|
|  ham|"SYMPTOMS" when U...|           139|
|  ham|"Speak only when ...|            80|
|  ham|"The world suffer...|           129|
|  ham|"Wen u miss someo...|           143|
|  ham|&lt;#&gt;  am I t...|            45|
|  ham|&lt;#&gt;  great ...|            85|
|  ham|&lt;#&gt;  w jett...|            37|
|  ham|&lt;#&gt; , that'...|    

### Fit your Pipeline model to the training data

In [ ]:
pipeline_model=pipe.fit(train_df)

### Perform predictions on tests dataframe

In [ ]:

test_results = pipeline_model.transform(test_df)
test_results.show()

+-----+--------------------+--------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|class|                text|length_of_text|label|            Tokenize|           stopWords|     CountVectorizer|              TF-IDF|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  ham| came to look at ...|           103|  0.0|[, came, to, look...|[, came, look, fl...|(10725,[3,11,199,...|(10725,[3,11,199,...|(10726,[3,11,199,...|[-830.87448469919...|[1.0,1.0224238648...|       0.0|
|  ham| said kiss, kiss,...|           133|  0.0|[, said, kiss,, k...|[, said, kiss,, k...|(10725,[3,122,223...|(10725,[3,122,223...|(10726,[3,122,223...|[-569.42030802

### Print the schema of the prediction dataframe

In [ ]:
test_results.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length_of_text: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- Tokenize: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stopWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CountVectorizer: vector (nullable = true)
 |-- TF-IDF: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [ ]:
# default value of MulticlassClassificationEvaluator is F1score
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval=MulticlassClassificationEvaluator()
f1Score=eval.evaluate(test_results)
print("F1 Score = %s" % f1Score)

F1 Score = 0.9766357331415284


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>